<a href="https://colab.research.google.com/github/gmagannaDevelop/segnet/blob/mru_tests/example_notebooks/MultiResUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiResUNet
## DCI-Net
### Primera Implementación
#### Gustavo Magaña

\########################################################

In [0]:
# Comentado ya que no es necesario :
# !pip install tensorflow-gpu
# !pip install git+https://github.com/DCI-NET/segnet
# Peligro : rm -rf segnet/
# pip install git+https://github.com/gmagannaDevelop/segnet.git@mru_tests

In [0]:
# Corrí esto una vez, ahora me imagino que más bien debo verificar que esté al día.
# EDIT : En realidad cada vez que es creada una máquina virtual, se debe clonar el repositorio.
# !git clone https://github.com/gmagannaDevelop/MyMultiResUNet.git
# !git clone https://github.com/DCI-NET/segnet.git
# !git clone https://github.com/gmagannaDevelop/segnet.git
# !git clone --single-branch --branch mru_tests https://github.com/gmagannaDevelop/segnet.git

In [0]:
# Actualizar, mientras la máquina virtual esté funcionando:
# !cd MyMultiResUNet/ && git pull
#!cd segnet/ && git pull

In [0]:
#!cd segnet/ && git branch

In [0]:
#!cd segnet/ && git log | head -6

In [0]:
#!cd segnet/ && git log | head -6

In [12]:
### System-related
import sys
import os
#import importlib.util
###############################################################

### Machine learning specific
#import segmentation_models as sm
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
###############################################################

### In-Out
from skimage import io
import glob
###############################################################

### Numerical
import numpy as np
###############################################################

### Repo-specific (segnet)
#from segnet.train_segnet import train_segnet
#from segnet.segnet.train.train_segnet import train_segnet
#import segnet.segnet as segnet

### Data-related
from google.colab import drive
drive.mount('/content/drive/')
###############################################################

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
!ls segnet/

example_notebooks  README.md	     segnet	     setup.py
LICENSE		   requirements.txt  segnet_env.yml  tests


In [0]:
!ls "drive/My Drive/DCI-Net/Data"

'Class Labels of Dataset 1.csv'  'Dataset 1'	  'Dataset 2'	    LICENSE
'Class Labels of Dataset 2.csv'  'Dataset 1.png'  'Dataset 2.png'   README.rst


In [29]:
'train' in dir(segnet)

False

In [0]:
# Define some example hyperparameters
batch_size = 8
epochs = 50
steps_per_epoch=100

# Declare the paths to use (following the Keras convention)
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator
data_path = "general/path/to/images"
img_path = data_path + "augmented/images/path"
masks_path = data_path + "augmented/masks/path"
model_file = "path/to/save/model/unet_model.h5"

# Train the model!
history = train_unet(
    img_path,
    masks_path,
    batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    model_file=model_file,
)

In [0]:
model = mru.MultiResUNet()

In [0]:
model.fit()